**Load python libraries**

In [ ]:
%matplotlib inline
import boto3
import botocore
import datetime
import matplotlib.pyplot as plt
import os.path
import xarray as xr
import numpy as np
import requests
import netCDF4
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
from IPython.display import Image, display
from glob import glob
from modules.GOESR_functions import goes_lat_lon_reproj, get_s3_keys
from modules.plotter import loadCPT

import matplotlib
matplotlib.use("Agg")
from matplotlib import colors as c
import matplotlib.animation as animation

In [ ]:
filename_pattern = '/home/sbarc/students/nash/data/GOES17_ABI_L2_CMI13/OR_ABI*.nc'

GOES_files = glob(filename_pattern) # search files 
GOES_files.sort()
DS = xr.open_mfdataset(GOES_files, concat_dim='t')
long_name = DS.CMI.long_name
lats, lons = goes_lat_lon_reproj(DS)

# put reprojected GOES HT data into new dataset
DS_goes = xr.Dataset({'CMI': (['time', 'y', 'x'], DS.CMI.values)},
                     coords={'time': DS.t.values,
                            'lat': (['y', 'x'], lats),
                            'lon': (['y', 'x'], lons)})
DS_goes

In [ ]:
def drawmap(ax, x, y, VO, cmap, title):

    datacrs = ccrs.PlateCarree()  ## the projection the data is in
    mapcrs = ccrs.PlateCarree() ## the projection you want your map displayed in
    ax = fig.add_subplot(1,1,1, projection=mapcrs)
    ax.set_title(title, fontsize=14)
    ax.set_extent(ext, crs=mapcrs)
    # Add Border Features
    coast = ax.coastlines(linewidths=1.0, resolution='10m')
    ax.add_feature(states_provinces, edgecolor='k')
    ax.add_feature(cfeature.BORDERS)
    # Add grid lines
    gl = ax.gridlines(crs=datacrs, draw_labels=True,
                      linewidth=.5, color='black', alpha=0.5, linestyle='--')
    gl.xlocator = mticker.FixedLocator(np.arange(-140., -110., 4))
    gl.ylocator = mticker.FixedLocator(np.arange(24, 44, 2))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabels_top = False
    gl.ylabels_right = False
    # Add contour plot
    cs = ax.pcolormesh(lons, lats, VO, transform=datacrs, cmap=cmap, vmin=-103, vmax=84, zorder=1)
    return cs
    
def myanimate(i, ax, DS, cmap):
    ax.clear()
    VO = DS.CMI.values[i]- 273.15 ## Convert to Celsius
    x = DS.lat
    y = DS.lon
    title = 'GOES-17 {0} at {1}'.format(long_name, DS_goes.time.values[i])
    new_contour = drawmap(ax,x,y,VO,cmap, title) 
    return new_contour

In [ ]:
lats = DS_goes.lat
lons = DS_goes.lon
VO = DS_goes.CMI.values - 273.15 ## Convert to Celsius


In [ ]:
# band_info = '(Band ' + str(DS.band_id.values[0]) + ', ' \
# + str(DS.band_wavelength.values[0]) + DS.band_wavelength.units + ')'
# title = 'GOES-17 {0} {1} on {2}'.format(DS.CMI.long_name, band_info, DS.time_coverage_end)

title = 'GOES-17 {0} at {1}'.format(long_name, DS_goes.time.values[0])
FFMpegWriter = animation.writers['ffmpeg']
metadata = dict(title=title,
                comment='Movie for GOES-17 Cloud Brightness Temp starting at 4 March 2019 18 UTC')
writer = FFMpegWriter(fps=20, metadata=metadata)

datacrs = ccrs.PlateCarree()  ## the projection the data is in
mapcrs = ccrs.PlateCarree() ## the projection you want your map displayed in
ext = [-115.0, -135.0, 25.0, 40.0]

# Import cmap from CPT
cmap = loadCPT('/home/sbarc/students/nash/data/Colortables/IR4AVHRR6.cpt')

# Create a feature for States/Admin 1 regions at 1:50m from Natural Earth
states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='50m',
    facecolor='none')

fig = plt.figure(figsize=[16,12])  # a new figure window
ax = fig.add_subplot(1,1,1, projection=mapcrs)
ax.set_extent(ext, crs=mapcrs)
# Add Border Features
coast = ax.coastlines(linewidths=1.0, resolution='10m')
ax.add_feature(states_provinces, edgecolor='k')
ax.add_feature(cfeature.BORDERS)
# Add grid lines
gl = ax.gridlines(crs=datacrs, draw_labels=True,
                  linewidth=.5, color='black', alpha=0.5, linestyle='--')
gl.xlocator = mticker.FixedLocator(np.arange(-140., -110., 4))
gl.ylocator = mticker.FixedLocator(np.arange(24, 44, 2))
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabels_top = False
gl.ylabels_right = False
first_contour = ax.pcolormesh(lons, lats, VO[0], transform=datacrs, cmap=cmap, vmin=-103, vmax=84)
ax.set_title(title)

# Add a color bar
cbar = fig.colorbar(first_contour, orientation='vertical', cmap=cmap, shrink=0.7)
Unit = "Brightness Temperature ($\mathrm{\degree C}$)"
cbar.set_label(Unit, fontsize=12)

ani = animation.FuncAnimation(fig, myanimate, frames=np.arange(len(DS_goes.CMI)), fargs=(ax, DS_goes, cmap), interval=50)
ani.save("GOES17_CMI.mp4")

In [ ]:
len(DS_goes.CMI)